# 16. BeautifulSoup 으로 Webpage 읽어오기

- 16_index.html file 을 webserver 에서 open   

    VS Code --> LiveServer 확장 프로그램 설치 --> Open with Live Server

## Web Scraping

- program 이 browser 인 것 처럼 행동하여 web page 를 access 하고 정보 추출  


    - hard way : regular expression 을 이용  
    - easy way : beautifulsoup 사용
    
- ``pip install beautifulsoup4``

In [1]:
from bs4 import BeautifulSoup
import urllib.request as req

### html file 읽기

- local web server start

In [4]:
url = "http://127.0.0.1:8887/23_index.html"

In [5]:
res = req.urlopen(url)

print(res.read().decode('utf-8'))

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>방송기술교육원</title>
</head>
<body>
    <h1>BeautifulSoup Web Crawling</h1>
    <p>뷰티플숩을 이용한 웹 크롤링</p>
    <div class="firstclass" id="div1">
        <p>첫번째 div 안의 paragraph 입니다.</p>
    </div>
    <div class="secondclass" id="div2">
        <p>두번째 div 안의 paragraph 입니다.</p>
    </div>
</body>
</html>


## html 내의 tag 내용 읽기

### hard way

- regular expression을 이용하여 html 내의 tag 검색

In [6]:
import re

res = req.urlopen(url)
text = res.read().decode('utf-8')

# web page 의 title 읽기
p1 = re.search('<title>', text)
p2 = re.search('</title>', text)

text[p1.span()[1] : p2.span()[0]]

'방송기술교육원'

### Easy way

- Beautifulsoup 이용

In [7]:
res = req.urlopen(url)

soup = BeautifulSoup(res, 'html.parser')

In [8]:
soup.title

<title>방송기술교육원</title>

In [9]:
print(soup.title.get_text())
print(soup.title.text)

방송기술교육원
방송기술교육원


- ``h1`` tag 내용 읽기

In [10]:
print(soup.h1)
print(soup.h1.text)

<h1>BeautifulSoup Web Crawling</h1>
BeautifulSoup Web Crawling


- 같은 tag 가 중복된 경우(ex. `div`) 첫번째 tag display

```
<body>
    <h1>BeautifulSoup Web Crawling</h1>
    <p>뷰티플숩을 이용한 웹 크롤링</p>
    <div class="firstclass" id="div1">
        <p>첫번째 div 안의 paragraph 입니다.</p>
    </div>
    <div class="secondclass" id="div2">
        <p>두번째 div 안의 paragraph 입니다.</p>
    </div>
</body>
```

In [11]:
soup.div

<div class="firstclass" id="div1">
<p>첫번째 div 안의 paragraph 입니다.</p>
</div>

- 중복된 tag 를 모두 find

In [12]:
soup.find_all('div')

[<div class="firstclass" id="div1">
 <p>첫번째 div 안의 paragraph 입니다.</p>
 </div>,
 <div class="secondclass" id="div2">
 <p>두번째 div 안의 paragraph 입니다.</p>
 </div>]

- 특정 id 를 지정하여 find

In [13]:
print(soup.find_all(id="div1"))
print(soup.find_all(id="div1")[0].text)

[<div class="firstclass" id="div1">
<p>첫번째 div 안의 paragraph 입니다.</p>
</div>]

첫번째 div 안의 paragraph 입니다.



- 특정 class 를 지정하여 find  
- class 는 Python 의 keyword 이므로 class_ 로 표시

In [14]:
soup.find_all(class_="secondclass")

[<div class="secondclass" id="div2">
 <p>두번째 div 안의 paragraph 입니다.</p>
 </div>]

- tag 내의 속성 (attribute) 가져오기

In [15]:
soup.find('div')['class']

['firstclass']

## 실전 연습

### `방송기술 교육원 ` 게시판 제목 검색

In [115]:
url = "http://edu.kobeta.com/community/"
res = req.urlopen(url)

# print(res.read().decode('utf-8'))

soup = BeautifulSoup(res, 'html.parser')

In [116]:
divs = soup.find_all(class_="notice-list-middle")

In [117]:
for div in divs:
    for dd in div.find_all('dd'):
        if re.search('[가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', dd.text):
            print(dd.text)

2021 콘텐츠 QC 세미나 개최
위탁교육 홍보 및 안내
지역 방송기술 세미나 희망 지역사 의견 접수
2021년도 컬러 그레이딩 전문가 양성과정 선발 결과 안내
방송 네트워크 파일기반 제작 시스템 교육 일정 연기 안내
5G Broadcast World Forum
[외부교육 안내] 2020 저널리즘아카데미 위드 코로나, 변화의 키워드 …
2020 Total Production Control System 전문가 양성과정 신청 안내
2019 글로벌 IP Technical 전문가 양성과정 신청 안내
2019 글로벌 IP Technical 전문가 양성과정 선발 결과 안내
2019 글로벌 Broadcast ATSC 3.0 전문가 양성과정 선발 결과 안내
2019 글로벌 Broadcast ATSC 3.0 전문가 양성과정 신청 안내
KOC 2018 "연결이 가치를 만들어낸다" 개최 안내


- 2nd page

In [119]:
url = "http://edu.kobeta.com/community/index.php?bbs_data=c3RhcnRQYWdlPTEwJmNvZGU9bm90aWNlJnRhYmxlPWNzX2Jic19kYXRhJnNlYXJjaF9pdGVtPSZzZWFyY2hfb3JkZXI9||"

In [121]:
res = req.urlopen(url)
soup = BeautifulSoup(res, 'html.parser')
divs = soup.find_all(class_="notice-list-middle")
for div in divs:
    for dd in div.find_all('dd'):
        if re.search('[가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', dd.text):
            print(dd.text)

2021 콘텐츠 QC 세미나 개최
위탁교육 홍보 및 안내
지역 방송기술 세미나 희망 지역사 의견 접수
2018 글로벌 UHD 실감형 오디오 제작 전문가 양성과정 선발 결과 안내
2018 글로벌 UHD 실감형 오디오 제작 전문가 양성과정 신청 안내
2018 글로벌 HDR 컬러리스트 전문가 양성과정 선발 결과 안내
2018 글로벌 HDR 컬러리스트 전문가 양성과정 신청 안내
KOBA World Media Forum 2018 개최 안내
2017 글로벌 방송제작기술 전문가 양성과정 선발 결과 안내
2017 글로벌 방송제작기술 전문가 양성과정 신청 안내
2017 'UHD 송신시스템(2차)' 폐강 및 '차세대 라디오 및 모바일 방송(2차…
2017년 9월 교육 과정 'UHD 송신시스템(2차)' 잠정 연기 안내
2017 글로벌 뉴미디어 전문가 양성과정 선발 결과 안내
